####  Run this cell to set up and start your interactive session.


In [ ]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
bucket_name = "BUCKET_NAME"
lines = sc.textFile(bucket_name)
lines.take(5)

In [ ]:
from pyspark.sql import Row

def mapper(text):
    fields = text.split(",")
    return Row(
        ID=int(fields[0]), 
        name=str(fields[1]).encode("utf-8"),
        age = int(fields[2]),
        numFriends = int(fields[3])
    )

people = lines.map(mapper)
people.take(5)


In [ ]:
schemaPeople = spark.createDataFrame(people).cache()
schemaPeople.createOrReplaceTempView("people")

In [ ]:
%%sql
SELECT * FROM people WHERE age >= 13 AND age <=19

In [ ]:
teenagers = spark.sql("SELECT * FROM people WHERE age >= 13 AND age <=19")
for teen in teenagers.collect():
    print(teen)

In [ ]:
schemaPeople.groupBy("age").count().orderBy("age").show()